In [9]:
!pip install playwright
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.10/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Registry._validateHostRequirements (/usr/

In [10]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
from google.colab import drive
import time
import re  # 正規表現ライブラリのインポート

# Colabでのイベントループの再利用を可能にする
nest_asyncio.apply()

# URLからコードと歌詞を取得する関数
async def fetch_lyrics_and_chords(url):
    try:
        async with async_playwright() as p:
            # ヘッドレスモードでChromiumブラウザを起動
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()

            # タイムアウトを90秒に設定して指定されたURLにアクセス
            await page.goto(url, timeout=90000)

            # ユーザーエージェントをセットする
            await page.set_extra_http_headers({
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
            })

            # preタグ内のすべてのテキストを取得
            pre_tag = await page.query_selector('pre')
            full_text = await pre_tag.inner_text()

            await browser.close()

            # コード部分と歌詞部分を分離
            # ここではコードが大文字アルファベットのみ、またはアルファベットと数字の組み合わせとして認識されると仮定
            # 正規表現でコードのパターンに一致する部分を抽出
            chords = re.findall(r'\b[A-G][#b]?(?:maj|maj7|min|dim|aug|sus|sus2|sus4|add9|m|m7|M7|m/A|7|6|9|13)?\b', full_text)

            chords = [chord for chord in chords if not re.match(r'Capo|\d+', chord)]

            # コード部分を取り除いた歌詞を作成
            lyrics = full_text
            # 「CHORDS:」セクションとそれに続く行を削除
            lyrics = re.sub(r'CHORDS:.*?(?=\n\n|\Z)', '', lyrics, flags=re.DOTALL)
            for chord in chords:
                # コードを歌詞から正確に削除する
                lyrics = re.sub(r'\b' + re.escape(chord) + r'\b', '', lyrics)


            # ギタータブ譜を削除 (行の先頭が e|, B|, G|, D|, A|, E| で始まる行を削除)
            lyrics = re.sub(r'^[eBGDAE]\|.*$', '', lyrics, flags=re.MULTILINE)
            # Intro: の前のテキストを削除
            lyrics = re.sub(r'.*?(?=Intro:)', '', lyrics, flags=re.DOTALL)
            # [Intro]以前のテキストを削除
            lyrics = re.sub(r'.*?(?=\[Intro\])', '', lyrics, flags=re.DOTALL)
            # [Outro]以降のすべてのテキストを削除
            lyrics = re.sub(r'\[Outro.*?\]', '', lyrics)
            lyrics = re.sub(r'Outro.*', '', lyrics)
            # []で囲まれた部分を削除
            lyrics = re.sub(r'\[.*?\]', '', lyrics)
            # Intro:, Verse 1:, Interlude: の削除
            lyrics = re.sub(r'(Intro|Verse \d+|Interlude|Pre-chorus|Chorus \d+|Outro):', '', lyrics)
            # 括弧内のテキストを削除
            lyrics = re.sub(r'\(.*?\)', '', lyrics)
            # ハイフンでつながった数字の部分（3-5-5や8-8）を削除
            lyrics = re.sub(r'\d+-\d+(?:-\d+)?', '', lyrics)
            # ハイフンを削除
            lyrics = lyrics.replace('-', '')
            # 「x4½」などの部分を削除
            lyrics = re.sub(r'x\d+½?', '', lyrics)
            # ピリオドで終わる文を削除
            lyrics = re.sub(r'.*?\.', '', lyrics)
            # "X" を削除
            lyrics = re.sub(r'\bX\b', '', lyrics)
            # "N.C." を削除
            lyrics = re.sub(r'\bN\.C\.\b', '', lyrics)
            # スラッシュ "/" と "|" を削除
            lyrics = re.sub(r'[\/|]', '', lyrics)
            # "#" を削除
            lyrics = re.sub(r'\b#\b', '', lyrics)
            # "%" を削除
            lyrics = re.sub(r'\b%\b', '', lyrics)
            # 数字を削除
            lyrics = re.sub(r'\d+', '', lyrics)
            # 余分なスペースや改行を整える
            lyrics = re.sub(r'\s+', ' ', lyrics).strip()

            return lyrics, ' '.join(chords)
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, None

async def main():
    # Google Driveのマウント
    drive.mount('/content/drive')

    # 曲のURLリストが保存されたテキストファイルのパスを指定
    file_path = '/content/drive/My Drive/music.txt'

    # テキストファイルを開く
    with open(file_path, 'r') as file:
        urls = file.read().splitlines()

    # URL、歌詞、コードのデータを格納するリスト
    data = []

    # 各URLについて、fetch_lyrics_and_chords関数を呼び出して、歌詞とコードを取得する
    for url in urls:
        lyrics, chords = await fetch_lyrics_and_chords(url)

        # 歌詞とコードが取得できた場合、それらをdataリストに追加する
        if lyrics and chords:
            data.append([url, lyrics, chords])
        else:
            print(f"Failed to fetch data for {url}")

    if data:
        # データをDataFrameに変換
        df = pd.DataFrame(data, columns=['URL', 'Lyrics', 'Chords'])

        # CSVファイルに保存
        df.to_csv('/content/drive/My Drive/lyrics_and_chords_separated.csv', index=False, encoding='utf-8')
        print("Data successfully saved to CSV.")
    else:
        print("No data to save.")

# 既存のイベントループで実行
await main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/chords.txt'

In [12]:
import nest_asyncio
import asyncio
from playwright.async_api import async_playwright
import csv
from google.colab import drive
import re

# Colabでのイベントループの再利用を可能にする
nest_asyncio.apply()

# URLからコードと歌詞を取得する関数
async def fetch_lyrics_and_chords(url):
    try:
        async with async_playwright() as p:
            # ヘッドレスモードでChromiumブラウザを起動
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()

            # タイムアウトを90000に設定して指定されたURLにアクセス
            await page.goto(url, timeout=90000)

            # ユーザーエージェントをセットする
            await page.set_extra_http_headers({
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
            })

            # preタグ内のすべてのテキストを取得
            pre_tag = await page.query_selector('pre')
            full_text = await pre_tag.inner_text()

            await browser.close()

            # コード部分と歌詞部分を分離
            chords = re.findall(r'\b[A-G][#b]?(?:maj|maj7|min|dim|aug|sus|sus2|sus4|add9|m|m7|M7|m/A|7|6|9|13)?\b', full_text)
            chords = [chord for chord in chords if not re.match(r'Capo|\d+', chord)]

            # コード部分を取り除いた歌詞を作成
            lyrics = full_text
            lyrics = re.sub(r'CHORDS:.*?(?=\n\n|\Z)', '', lyrics, flags=re.DOTALL)
            for chord in chords:
                lyrics = re.sub(r'\b' + re.escape(chord) + r'\b', '', lyrics)

            # ギタータブ譜や不要な部分を削除
            lyrics = re.sub(r'^[eBGDAE]\|.*$', '', lyrics, flags=re.MULTILINE)
            lyrics = re.sub(r'.*?(?=Intro:)', '', lyrics, flags=re.DOTALL)
            lyrics = re.sub(r'.*?(?=\[Intro\])', '', lyrics, flags=re.DOTALL)
            lyrics = re.sub(r'\[Outro.*?\]', '', lyrics)
            lyrics = re.sub(r'Outro.*', '', lyrics)
            lyrics = re.sub(r'\[.*?\]', '', lyrics)
            lyrics = re.sub(r'(Intro|Verse \d+|Interlude|Pre-chorus|Chorus \d+|Outro):', '', lyrics)
            lyrics = re.sub(r'\(.*?\)', '', lyrics)
            lyrics = re.sub(r'\d+-\d+(?:-\d+)?', '', lyrics)
            lyrics = lyrics.replace('-', '')
            lyrics = re.sub(r'x\d+½?', '', lyrics)
            lyrics = re.sub(r'.*?\.', '', lyrics)
            lyrics = re.sub(r'\bX\b', '', lyrics)
            lyrics = re.sub(r'\bN\.C\.\b', '', lyrics)
            lyrics = re.sub(r'[\/|]', '', lyrics)
            lyrics = re.sub(r'#', '', lyrics)
            lyrics = re.sub(r'\b%\b', '', lyrics)
            lyrics = re.sub(r'\d+', '', lyrics)
            lyrics = re.sub(r'\s+', ' ', lyrics).strip()

            # 歌詞とコードのどちらも取得できるか確認
            status = 1 if lyrics and chords else 0

            return lyrics, ' '.join(chords), status
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, None, 0

async def main():
    # Google Driveのマウント
    drive.mount('/content/drive')

    # 曲のURLリストが保存されたテキストファイルのパスを指定
    file_path = '/content/drive/My Drive/chords.txt'
    output_file = '/content/drive/My Drive/lyrics_and_chords_separated3.csv'

    # CSVファイルにヘッダーを書き込む（ファイルが存在しない場合）
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['URL', 'Lyrics', 'Chords', 'Status'])

    # テキストファイルを開く
    with open(file_path, 'r') as file:
        urls = file.read().splitlines()

    # 各URLについて、fetch_lyrics_and_chords関数を呼び出して、歌詞とコードを取得する
    for url in urls:
        lyrics, chords, status = await fetch_lyrics_and_chords(url)

        # 取得したデータをCSVに追加
        with open(output_file, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([url, lyrics, chords, status])

        # ステータスの確認
        if status == 1:
            print(f"Data for {url} saved successfully.")
        else:
            print(f"Failed to fetch data for {url}")

# 既存のイベントループで実行
await main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error fetching https://tabs.ultimate-guitar.com/tab/lady-gaga/die-with-a-smile-chords-5389398: Page.goto: Timeout 90000ms exceeded.
Call log:
navigating to "https://tabs.ultimate-guitar.com/tab/lady-gaga/die-with-a-smile-chords-5389398", waiting until "load"

Failed to fetch data for https://tabs.ultimate-guitar.com/tab/lady-gaga/die-with-a-smile-chords-5389398
Data for https://tabs.ultimate-guitar.com/tab/gigi-perez/sailor-song-chords-5363001 saved successfully.
Data for https://tabs.ultimate-guitar.com/tab/gracie-abrams/i-love-you-im-sorry-chords-5315181 saved successfully.
Data for https://tabs.ultimate-guitar.com/tab/elvis-presley/cant-help-falling-in-love-chords-1086983 saved successfully.
Data for https://tabs.ultimate-guitar.com/tab/radiohead/creep-chords-4169 saved successfully.
Data for https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1